In [1]:
import torch
import torch.nn as nn
from sklearn.datasets import fetch_olivetti_faces

In [2]:
# 从sklearn中获取olivetti_faces数据集
olivetti_faces = fetch_olivetti_faces(data_home='D:\\datasets\\face_data', shuffle=True) 

In [6]:
images = torch.tensor(olivetti_faces.data)
targets = torch.tensor(olivetti_faces.target)
dataset = [(img,lbl) for img,lbl in zip(images, targets)]
dataset[0]

(tensor([0.6694, 0.6364, 0.6488,  ..., 0.0868, 0.0826, 0.0744]),
 tensor(13, dtype=torch.int32))

In [38]:
dataloaders = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

In [39]:
epochs = 10
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [40]:
model = nn.Sequential(
    nn.Linear(4096,8192),
    nn.BatchNorm1d(8192),
    nn.ReLU(),
    nn.Linear(8192,4096),
    nn.BatchNorm1d(4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(2048,1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1024,40)
).to(device)

In [41]:
loss_fn = nn.CrossEntropyLoss()  # 交叉熵损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam优化器

In [46]:
loss_hist = [] # To store the loss history
for epoch in range(epochs): # Run the loop for epochs
    for img, lbl in dataloaders:
        img, lbl = img.to(device), lbl.to(device)
        # Forward pass
        result = model(img)
        loss = loss_fn(result, lbl.long())
        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # Store values
        loss_hist.append(loss.item())
        # Print the loss value every 10 epochs  
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/10], Loss: 0.4843
Epoch [10/10], Loss: 0.5842
Epoch [10/10], Loss: 0.2958
Epoch [10/10], Loss: 0.3488
Epoch [10/10], Loss: 0.3895
Epoch [10/10], Loss: 0.1882
Epoch [10/10], Loss: 0.7694
Epoch [10/10], Loss: 0.3109
Epoch [10/10], Loss: 0.3583
Epoch [10/10], Loss: 0.4001
Epoch [10/10], Loss: 0.6342
Epoch [10/10], Loss: 0.6132
Epoch [10/10], Loss: 0.4207
Epoch [10/10], Loss: 0.3427
Epoch [10/10], Loss: 1.1243
Epoch [10/10], Loss: 0.2599
Epoch [10/10], Loss: 0.5135
Epoch [10/10], Loss: 0.2195
Epoch [10/10], Loss: 0.9320
Epoch [10/10], Loss: 0.2512
Epoch [10/10], Loss: 0.1887
Epoch [10/10], Loss: 0.7291
Epoch [10/10], Loss: 0.3230
Epoch [10/10], Loss: 0.2922
Epoch [10/10], Loss: 0.6889
Epoch [10/10], Loss: 0.0823
Epoch [10/10], Loss: 0.2152
Epoch [10/10], Loss: 0.4442
Epoch [10/10], Loss: 0.9303
Epoch [10/10], Loss: 0.3252
Epoch [10/10], Loss: 0.1935
Epoch [10/10], Loss: 1.2266
Epoch [10/10], Loss: 1.2575
Epoch [10/10], Loss: 1.7672
Epoch [10/10], Loss: 1.4055
Epoch [10/10], Loss:

In [47]:
#测试
test_loader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

correct = 0
total = 0
with torch.no_grad():  # 测试集不需要反向传播
    for data, target in test_loader:
        outputs = model(data.reshape(-1, 4096))
        _, predicted = torch.max(outputs, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

# output.shape
# output
print('Accuracy of the network : %d %%' % (100 * correct / total))
print(f'predicted', predicted)
print(f'target', target)
print(f'num: {(predicted == target).int().sum()}')
print(f'Accuracy: {(predicted == target).int().sum().item() / len(target) * 100:.2f}%')

Accuracy of the network : 81 %
predicted tensor([11, 23,  3, 35, 18, 13, 25, 38, 17, 28])
target tensor([13, 23,  3,  5,  7, 13, 25, 38, 17, 28], dtype=torch.int32)
num: 7
Accuracy: 70.00%
